In [50]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import CIFAR10
from torchvision import transforms as tfs

In [163]:
# 这是定义残差模块
# 这些在torchvision/model/resnet.py都定义好了
# 造轮子失败！
# 可以参考代码： https://github.com/L1aoXingyu/code-of-learn-deep-learning-with-pytorch/blob/master/chapter4_CNN/utils.py
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False
    )
    
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, same_shape=True):
        super(ResidualBlock, self).__init__()
        self.same_shape = same_shape
        stride = 1 if self.same_shape else 2
        
        self.conv1 = conv3x3(in_channels, out_channels, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels, stride=stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel, 1, stride=stride)
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if not self.same_shape:
            x = self.conv3(x)
        
        out += residual
        out = self.relu(out)
        return out

In [168]:
# 定义残差网络. 
class ResNet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(ResNet, self).__init__()
        self.verbose = verbose

        self.block1 = nn.Conv2d(in_channel, 64, 7, 2)

        self.block2 = nn.Sequential(
            nn.MaxPool2d(3, 2), residual_block(64, 64), residual_block(64, 64))

        self.block3 = nn.Sequential(
            residual_block(64, 128, False), residual_block(128, 128))

        self.block4 = nn.Sequential(
            residual_block(128, 256, False), residual_block(256, 256))

        self.block5 = nn.Sequential(
            residual_block(256, 512, False),
            residual_block(512, 512), nn.AvgPool2d(3))

        self.classifier = nn.Linear(512, num_classes)

    
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x


In [169]:
# 把数据做增强处理。
# 先缩小
def data_tf(x):
    im_aug = tfs.Compose([
        # tfs.Resize(96),                                       # 图像短边缩小至96，图像长宽比不变
        tfs.ToTensor(),
        tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])       # 图像3个通道都转成 mean=0.5，std=0.5 的数据
    ])
    x = im_aug(x)
    return x

In [170]:
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [171]:
train_set = CIFAR10('./data', train=True, transform=data_tf, download=True)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf, download=False)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

model = ResNet(ResidualBlock, [2,2,2])
learning_rate = 1e-3
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, weight_decay=1e-4) # 增加正则项
lossfunc = nn.CrossEntropyLoss()

Files already downloaded and verified


TypeError: unsupported operand type(s) for %: 'type' and 'int'

In [ ]:
total_step = len(train_data)
num_epoches = 20

for epoch in range(num_epoches):
    for i, (img, label) in enumerate(train_data):
        img = Variable(img)
        label = Variable(label)
        
        output = model(img)
        loss = lossfunc(img, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1%100==0):
            print('epoch:{} step:{} loss{}'.format(epoch, i, loss.item()))